In [1]:
# 출처 : https://ncov.kdca.go.kr/bdBoardListR.do?brdId=1&brdGubun=11
# 코로나바이러스감염증-19_확진환자_발생현황_230904_최종v2.xlsx : '연령별(10세단위)' sheet상의 2020.01.20~2023.08.31기간 동안의 일별 연령별 확진자에 대한 데이터 사용
# dataframe[df_c_raw]로 불러와 월별로 확진자를 합산한 dataframe[df_confirmed_case]으로 정리하기

In [2]:
# pandas패키지 사용하기
import pandas as pd
import numpy as np

#경고 뜨지 않게 하기
import warnings
warnings.filterwarnings(action='ignore')

# xlsx파일(코로나바이러스감염증-19_확진환자_발생현황_230904_최종v2.xlsx)의 원하는 특정시트sheet('연령별(10세단위)')에서 dataframe으로 읽어오기
df_c_raw = pd.read_excel('코로나바이러스감염증-19_확진환자_발생현황_230904_최종v2.xlsx', sheet_name='연령별(10세단위)')

# row[3]을 column이름으로 바꾸기 
df_c_tmp1 = df_c_raw.rename(columns = df_c_raw.iloc[3])

# 필요하지 않은 row[0]~[4] 데이터 지우기
df_c_tmp2 = df_c_tmp1.iloc[5:, :]

#str인 '-'를 int인 0으로 변경(데이터 가공 용이를 위함)
df_c_tmp3 = df_c_tmp2.replace(to_replace='-',value=0)

#2023-08-29 ~ 2023-08-31의 값이 누락되어 채워넣기
df_c_tmp3.loc[1322, '일자'] = '2023-08-29 00:00:00'
df_c_tmp3.loc[1323, '일자'] = '2023-08-30 00:00:00'
df_c_tmp3.loc[1324, '일자'] = '2023-08-31 00:00:00'

# column['일자']의 데이터에서 년월만 떼어낸 year_month데이터를 통해 월별로 묶어서(groupby) 확진자수합계(sum)데이터 만들기
year_month = df_c_tmp3.일자.astype('str').str.slice(0,7)  ## str타입이 아니어서 astype('str')을 해주고 strslice(0,7)을 통해 년월일시 데이터를 년월로 자름
df_c_tmp4 = df_c_tmp3.groupby(year_month).sum()  ## 년월데이터(year_month)를 통해 groupby하여 합산
## 이때 groupby로 인해 year_month 데이터는 해당 dataframe의 인덱스가 된다.

#년월데이터(index)에서 str.slice()을 통해 column[year], column[month]를 추가하기 
year = df_c_tmp4.index.str.slice(0,4)
month = df_c_tmp4.index.str.slice(5,7)
df_c_tmp4['year'] = year
df_c_tmp4['month'] = month

# reset_index()를 통해 dataframe의 index를 0,1,2,...로 변경하기
df_c_tmp5 = df_c_tmp4.reset_index()

# dataframe의 column명을 원하는대로 바꾸기
df_c_tmp5.columns = ['일자', '계(명)', '0-9(confirmed_case)', '10-19(confirmed_case)', '20-29(confirmed_case)', 
                    '30-39(confirmed_case)', '40-49(confirmed_case)','50-59(confirmed_case)', '60-69(confirmed_case)', 
                    '70-79(confirmed_case)', 'over80(confirmed_case)', 'year', 'month']

#필요하지 않은 column[0]~[2] 데이터 지우기
df_c_tmp6=df_c_tmp5.iloc[:, 2:]

# dataframe의 column 순서(및 구성) 변경하기
df_confirmed_case = df_c_tmp6[['year','month', '0-9(confirmed_case)', '10-19(confirmed_case)', '20-29(confirmed_case)',
       '30-39(confirmed_case)', '40-49(confirmed_case)',
       '50-59(confirmed_case)', '60-69(confirmed_case)',
       '70-79(confirmed_case)', 'over80(confirmed_case)']]

In [4]:
# 출처 : https://jumin.mois.go.kr
# 2020년 01월 ~ 2023년 08월 동안의 월별 전국 연령별 인구수에 대한 데이터를 사용 
# 위 기간이 기간별로 8개의 파일로 분리되어 있는데 이를 dataframe[df_p_raw_p1]~[df_p_raw_p8]로 불러와 월별로 전국 연령별 인구수를 dataframe[df_population]으로 정리하기

In [5]:
# 8개의 csv을 dataframe으로 읽어오기
## 유니코드(UTF-8)로 코딩이 되지 않아 한국 코드(euc-kr)를 사용하도록 함.
## str로 가져오기 때문에 thousands = ','을 통해 ','없이 데이터를 가져와 가공하기 쉽게한다.
df_p_raw1= pd.read_csv('202001_202002_연령별인구현황_월간.csv', encoding = 'euc-kr', thousands = ',')
df_p_raw2= pd.read_csv('202003_202008_연령별인구현황_월간.csv', encoding = 'euc-kr', thousands = ',')
df_p_raw3= pd.read_csv('202009_202102_연령별인구현황_월간.csv', encoding = 'euc-kr', thousands = ',')
df_p_raw4= pd.read_csv('202103_202108_연령별인구현황_월간.csv', encoding = 'euc-kr', thousands = ',')
df_p_raw5= pd.read_csv('202109_202202_연령별인구현황_월간.csv', encoding = 'euc-kr', thousands = ',')
df_p_raw6= pd.read_csv('202203_202208_연령별인구현황_월간.csv', encoding = 'euc-kr', thousands = ',')
df_p_raw7= pd.read_csv('202209_202302_연령별인구현황_월간.csv', encoding = 'euc-kr', thousands = ',')
df_p_raw8= pd.read_csv('202303_202308_연령별인구현황_월간.csv', encoding = 'euc-kr', thousands = ',')

# 8개의 csv파일을 concat을 통해 상하 합치기(axis=0) 
## ignore_index=False를 통해 컬럼명이나 인덱스를 그대로 유지해 가져옴
concat_list = [df_p_raw1, df_p_raw2, df_p_raw3, df_p_raw4, df_p_raw5, df_p_raw6, df_p_raw7, df_p_raw8]
df_p_tmp_1 = pd.concat(concat_list, axis=0, ignore_index=False)

# df_p_tmp_1['행정구역'] == '전국  (0000000000)' bool자료를 통해 필요치 않은 지역별 연령별인구현황 row를 제거
df_p_tmp_2 = df_p_tmp_1.loc[df_p_tmp_1['행정구역'] == '전국  (0000000000)']

# NaN 데이터를 0으로 바꾸기
df_p_tmp_3 = df_p_tmp_2.replace(np.nan,0)

# dataframe의 행렬 전환
df_p_tmp_4 = df_p_tmp_3.T

# 필요하지 않은 row[0] 데이터 지우기
df_p_tmp_5 = df_p_tmp_4.iloc[1:,:]

# 년,월,연령구간 데이터가 있는 index에서 str.slice()를 통해 column[year], column[month], column[step] 데이터 만들기
year = df_p_tmp_5.index.astype('str').str.slice(0,4) 
month = df_p_tmp_5.index.astype('str').str.slice(5,7)
step = df_p_tmp_5.index.astype('str').str[9:]
df_p_tmp_5['year'] = year
df_p_tmp_5['month'] = month
df_p_tmp_5['step'] = step

# 여러행에 나눠져있는 전국 연령별 인구수를 합쳐(sum) 정리한 column[total] 데이터 만들기
total = df_p_tmp_5.iloc[:,:8].astype(int).sum(axis =1)
df_p_tmp_5['total'] = total.astype(int)

#피봇테이블을 할 데이터만 남기기
df_p_tmp_6 = df_p_tmp_5.loc[:, ['year', 'month', 'step', 'total']]

# column[year], column[month]을 index(행)로 하여 column[step]별로 인구수를 표현하는 피봇테이블 생성
df_p_tmp_7 = df_p_tmp_6.pivot(index=['year','month'],columns=['step'],values=['total'])

# reset_index()를 통해 index가 된 column[year], column[month]을 column으로 변경하기
df_p_tmp_8 = df_p_tmp_7.reset_index()

# multiindex인 column명을 droplevel(0)을 통해 index로 바꾸기
df_p_tmp_8.columns = df_p_tmp_8.columns.droplevel(0)

# 필요하지 않은 데이터 제거하기
df_p_tmp_9 = df_p_tmp_8.iloc[:,0:13]

# 컬럼명 바꾸기
df_p_tmp_9.columns =  np.concatenate([np.array(['year','month']),df_p_tmp_9.columns[2:]])

# 80세 이상인 column을 모아 합계를 구하고, 이전 데이터는 삭제
df_p_tmp_9['80세이상'] = df_p_tmp_9.iloc[:,[3,11,12]].sum(1)
df_p_tmp_9.drop(columns=df_p_tmp_9.iloc[:,[3,11,12]].columns,inplace=True)

# 컬럼명 바꾸기
df_p_tmp_9.columns = ['year','month', '0-9(population)', '10-19(population)', '20-29(population)',
       '30-39(population)', '40-49(population)', '50-59(population)',
       '60-69(population)', '70-79(population)', 'over80(population)']

# dataframe 이름 바꾸기
df_population = df_p_tmp_9

In [7]:
# 년월별에 따른 연령마다의 확진률(확진자수/총인구수) : df_r_raw
# 가공하여 df_rate를 만들기

In [8]:
# dataframe끼리의 '/'을 통해 df_r_raw 생성하기
df_r_raw = pd.DataFrame(df_confirmed_case.values.astype(int)/df_population.values.astype(int),columns=df_population.columns)

# 필요치 않은 데이터 제거
df_r_tmp1 = df_r_raw.iloc[:, 2:]

# 컬럼명 바꾸기
df_r_tmp1.columns = ['0-9(rate)', '10-19(rate)', '20-29(rate)',
       '30-39(rate)', '40-49(rate)',
       '50-59(rate)', '60-69(rate)',
       '70-79(rate)', 'over80(rate)']

#데이터프레임 이름바꾸기
df_rate = df_r_tmp1

In [9]:
# dataframe[df_population], dataframe[df_confirmed_case], dataframe[df_rate]을 합치고, 원하는 컬럼순대로 정리 후, csv로 내보내기

In [10]:
# dataframe concat으로 합치기
df = pd.concat([df_population, df_confirmed_case, df_rate], axis=1)

# 컬럼순서 정리하기
df_final=df[['year', 'month', 
             '0-9(population)', '0-9(confirmed_case)', '0-9(rate)',
            '10-19(population)', '10-19(confirmed_case)', '10-19(rate)',
            '20-29(population)', '20-29(confirmed_case)', '20-29(rate)',
            '30-39(population)', '30-39(confirmed_case)', '30-39(rate)',
            '40-49(population)', '40-49(confirmed_case)', '40-49(rate)',
            '50-59(population)', '50-59(confirmed_case)', '50-59(rate)',
            '60-69(population)', '60-69(confirmed_case)', '60-69(rate)',
            '70-79(population)', '70-79(confirmed_case)', '70-79(rate)',
            'over80(population)', 'over80(confirmed_case)', 'over80(rate)']]

# csv파일로 내보내기
df_final.to_csv('df_final.csv')